# Данные
#### [Our World in Data website](https://ourworldindata.org/coronavirus-source-data)
#### [Our World in Data GitHub](https://github.com/owid/covid-19-data/tree/master/public/data/)

Вопрос европейского заказчика (dummy, но запрос валидный и вполне расплывчатый, чтобы сойти за настоящий):

***На какой рынок сейчас стоит выходить, если рассматривать ситуацию с COVID-19?***

Что нужно понять:
- Откуда брать данные
- Какие инструменты использовать
- Какие метрики хочет видеть заказчик
- Какой ответ на вопрос и/или на уточненный вопрос

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import files


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

plt.style.use('ggplot')
# sns.set_theme(style='darkgrid')

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Подгрузим данные

In [ ]:
df = pd.read_csv(
    "https://covid.ourworldindata.org/data/owid-covid-data.csv",
    parse_dates=['date'],
    low_memory=False,
)
df['date'] = df['date'].dt.date

# Ознакомимся с датасетом 

In [ ]:
df.info()

In [ ]:
df.head(15)

# Посмотрим на то, что нас интересует

In [ ]:
df['continent'].unique()

In [ ]:
df['location'].unique()

# Выберем то, что интересует

In [ ]:
europe = df[df['continent'] == 'Europe']

# Попробуем выделить наиболее значимые и простые столбцы / агрегаты
Можно использовать популярные методы `pandas.DataFrame`:
- `.groupby()`
- `.isna()`
- `.sum()`
- `.mean()`

In [ ]:
# посмотрите, сколько пропусков по столбцам для Европы
europe.isna().sum()

In [ ]:
# посмотрите, сколько в среднем новых заболевших в Европе в апреле
(
    europe
    [europe.date >= pd.to_datetime('2022-04-01').date()]
    .groupby('location')
    ['new_cases']
    .mean()
    .sort_values()
)

In [ ]:
# extra
# посмотрите, какой средний процент от населения составляют все случаи в апреле
gb = (
    europe
    [europe.date >= pd.to_datetime('2022-04-01').date()]
    .groupby('location')
)
gb.apply(lambda d: (d.total_cases / d.population * 100).mean())

# Построим наиболее значимые и очевидные метрики

In [ ]:
# всего случаев
sns.lineplot(
    x='date',
    y='total_cases',
    data=europe,
    ci=None,
)

In [ ]:
# новых случаев
sns.lineplot(
    x='date',
    y='new_cases',
    data=europe,
    ci=None,
)

In [ ]:
# всего смертей
sns.lineplot(
    x='date',
    y='total_deaths',
    data=europe,
    ci=None,
)

In [ ]:
# новых смертей
sns.lineplot(
    x='date',
    y='new_deaths',
    data=europe,
    ci=None,
)

# Что не так с этими графиками?
- умная мысль 1
- умная мысль 2
- гениальное заключение 3

## А теперь давайте попробуем построить график, который уже и показывать не слишком стыдно

In [ ]:
country = 'Russia'
country_df = df[df['location'] == country]

In [ ]:
# plotting country_df
fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(8, 14))
ax0.set_title(f'Cases and deaths of COVID-19 in {country} by date', y=1.05)
ax1.set_xlabel('Date')
red, blue = plt.cm.RdBu([0.01, 0.99])

# total cases
ax0.plot(country_df.date, country_df.total_cases, color=blue)
plt.setp(ax0.get_xticklabels(), rotation=45)

# new cases
ax2 = ax0.twinx()
ax2.plot(country_df.date, country_df.new_cases, color=red)
ax2.grid(None)

# total deaths
ax1.plot(country_df.date, country_df.total_deaths, color=blue)
plt.setp(ax1.get_xticklabels(), rotation=45)

# new deaths
ax3 = ax1.twinx()
ax3.plot(country_df.date, country_df.new_deaths, color=red)
ax3.grid(None)

# prettifying plots
for ax, ylabel, color in zip([ax0, ax1, ax2, ax3],
                                ['Total cases',
                                'Total deaths',
                                'New cases',
                                'New deaths'],
                                [blue, blue, red, red]):
    ax.set_ylabel(ylabel, color=color)
    ax.tick_params('y', colors=color)
    ax.set_ylim(0)
    ax.set_yticks(np.linspace(*ax.get_ybound(), 10))

plt.show(block=False)

# А что, если построить рядом сразу несколько стран?
Без проблем!

In [ ]:
countries = (
    'Russia',
    'United States',
    'United Kingdom',
    'Sweden',
)

In [ ]:
fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(13, 13))
ax0.set_title(f'Cases and deaths of COVID-19 in {country} by date', y=1.05)

sns.lineplot(
    x='date',
    y='total_cases',
    data=df[df['location'].isin(countries)],
    hue='location',
    ax=ax0,
)
ax0.set_ylabel('Total cases')
ax0.set_xlabel('')

sns.lineplot(
    x='date',
    y='new_cases',
    data=df[df['location'].isin(countries)],
    hue='location',
    ax=ax1,
)
ax1.set_ylabel('Total cases')
ax1.set_xlabel('Date');

# Давайте выгрузим часть данных для заказчика

In [ ]:
europe.columns

In [ ]:
cols = [
    'location',
    'date',
    'total_cases',
    'new_cases',
    'total_deaths',
    'new_deaths',
    'total_tests',
    'new_tests',
    'total_vaccinations',
    'people_vaccinated',
    'people_fully_vaccinated',
    'population',
    'median_age',
    'aged_65_older',
]
europe[europe.date >= pd.to_datetime('2022-01-01').date()].to_excel('covid.xlsx', index=False)
europe[europe.date >= pd.to_datetime('2022-01-01').date()].to_csv('covid.csv', index=False)

files.download('covid.xlsx')
files.download('covid.csv')